### Gezinomi - Otel Satış Fiyatlarının Modellenmesi

**İş Problemi**

Gezinomi'nin gerçekleştirdiği satış verilerini kullanarak, gelecekteki müşterilerin ortalama otel ücretini tahmin etmek istiyoruz.

**Örneğin:** İzmir'de yoğun bir dönemde yarım pansiyon bir otele gitmek isteyen bir müşterinin ödeyeceği tahmini ücreti belirlemek istiyoruz.

**Veri Seti Hikayesi**

**gezinomi_miuul.xslx** veri seti Gezinomi şirketinin yapmış olduğu satışların fiyatlarını ve bu satışlara ait bilgiler içermektedir. Veri seti her bir satış işleminde oluşan kayıtlardan meydana gelmektedir. Bunun anlamı tablolar tekilleştirilmemiştir. Diğer bir ifade ile her bir müşteri birden  fazla kez rezervasyon işlemi yapmış olabilir.

**Değişkenler**

**SaleID:** Satış id

**SaleDate:** Satış Tarihi

**CheckInDate:** Müşterinin otele giriş tarihi

**Price:** Satış için ödenen fiyat

**ConceptName:** Otel konsept bilgisi

**SaleCıtyName:** Otelin bulunduğu şehir bilgisi

**CInDay:** Müşterinin otele giriş günü

**SaleCheckInDayDiff:** Check in ile giriş tarihi gün farkı

**Season:** Otele girişindeki sezon bilgisi

#### Görev 1: Veri Setine Genel Bir Bakış

**Adım 1:** gezinomi_miuul.xlsx dosyasını okutunuz ve veri seti ile ilgili genel bilgileri gösteriniz.

In [1]:
# Gerekli Kütüphane ve Fonksiyonların Kurulması

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.float_format", lambda x: '%.2f' % x)

gezinomi_data =  pd.read_excel("dataset/gezinomi_miuul.xlsx")

df = gezinomi_data.copy()

df.head()

,SaleId,SaleDate,CheckInDate,Price,ConceptName,SaleCityName,CInDay,SaleCheckInDayDiff,Seasons
0,415122,2022-12-03,2022-12-03,79.30,Herşey Dahil,Antalya,Saturday,0,Low
1,415103,2022-12-03,2022-12-03,45.97,Yarım Pansiyon,Antalya,Saturday,0,Low
2,404034,2022-09-12,2022-09-13,77.84,Herşey Dahil,Antalya,Tuesday,1,High
3,415094,2022-12-03,2022-12-10,222.71,Yarım Pansiyon,İzmir,Saturday,7,Low
4,414951,2022-12-01,2022-12-03,140.48,Yarım Pansiyon,İzmir,Saturday,2,Low


In [2]:
df.tail()

,SaleId,SaleDate,CheckInDate,Price,ConceptName,SaleCityName,CInDay,SaleCheckInDayDiff,Seasons
59159,51817,2016-01-05,2016-10-10,54.30,Herşey Dahil,Antalya,Monday,279,Low
59160,51816,2016-01-05,2016-10-10,54.30,Herşey Dahil,Antalya,Monday,279,Low
59161,51814,2016-01-05,2016-01-06,40.56,Herşey Dahil,Diğer,Wednesday,1,Low
59162,51736,2016-01-04,2016-01-05,69.85,Yarım Pansiyon,Diğer,Tuesday,1,Low
59163,51731,2016-01-04,2016-08-22,158.94,Herşey Dahil,Antalya,Monday,231,High


In [3]:
df.shape

(59164, 9)

In [4]:
df.columns

Index(['SaleId', 'SaleDate', 'CheckInDate', 'Price', 'ConceptName',
       'SaleCityName', 'CInDay', 'SaleCheckInDayDiff', 'Seasons'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59164 entries, 0 to 59163
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   SaleId              59164 non-null  int64         
 1   SaleDate            59164 non-null  datetime64[ns]
 2   CheckInDate         59164 non-null  datetime64[ns]
 3   Price               59151 non-null  float64       
 4   ConceptName         59164 non-null  object        
 5   SaleCityName        59164 non-null  object        
 6   CInDay              59164 non-null  object        
 7   SaleCheckInDayDiff  59164 non-null  int64         
 8   Seasons             59164 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(2), object(4)
memory usage: 4.1+ MB


In [6]:
df.isnull().sum()

SaleId                 0
SaleDate               0
CheckInDate            0
Price                 13
ConceptName            0
SaleCityName           0
CInDay                 0
SaleCheckInDayDiff     0
Seasons                0
dtype: int64

In [7]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
SaleId,59164.00,302675.80,51707.00,274038.50,321607.00,367373.00,415122.00,87943.22
SaleDate,59164,2021-02-11 17:13:11.751740928,2016-01-02 00:00:00,2020-08-19 00:00:00,2021-07-28 00:00:00,2022-05-10 00:00:00,2022-12-03 00:00:00,NaN
CheckInDate,59164,2021-03-15 11:23:01.732134400,2007-07-20 00:00:00,2020-08-24 00:00:00,2021-08-16 00:00:00,2022-07-04 00:00:00,2023-08-16 00:00:00,NaN
Price,59151.00,61.35,0.00,35.34,54.26,77.96,4880.47,51.38
SaleCheckInDayDiff,59164.00,31.84,0.00,2.00,10.00,34.00,630.00,51.49


**Adım 2:** Boş değerlerin sayısı ile veri setinin büyüklüğünü kıyaslayın. Veri setinde önemli bir bilgi kaybı olmayacağı bir durum ise boş değerlerin bulunduğu gözlem birimlerini silin. 

In [8]:
df.dropna(inplace = True)

df.isnull().sum()

SaleId                0
SaleDate              0
CheckInDate           0
Price                 0
ConceptName           0
SaleCityName          0
CInDay                0
SaleCheckInDayDiff    0
Seasons               0
dtype: int64

#### Görev 2: Özellik Mühendisliği

**Adım 1:** Check-in tarihini kullanarak aşağıdaki değişkenleri türetiniz:

"CheckInYear:" Check-in yapıldığı yıl

"CheckInMonth:" Check-in yapıldığı ay

"CheckInDay:" Check-in yapıldığı gün

"CheckInWeekday:" Check-in yapıldığı gün haftanın kaçıncı günü

"IsWeekend:" Check-in yapıldığı günün hafta sonu olup olmadığı

"HolidayTypes:" Verisetindeki check-in yapılan en eski tarih ile en yeni tarih arasındaki Türkiye'deki resmi tatiller. Örneğin, 2016-08-30-Zafer Bayramı, 2022-07-09-Kurban Bayramı

In [9]:
# CheckInDate'den özellikler Çıkarma

df["CheckInYear"]    = df["CheckInDate"].dt.year
df["CheckInMonth"]   = df["CheckInDate"].dt.month
df["CheckInDay"]     = df["CheckInDate"].dt.day
df["CheckInWeekday"] = df["CheckInDate"].dt.dayofweek

# Günün hafta sonu olup olmadığının kontrolü

df["IsWeekend"] = np.where(df["CheckInWeekday"] >= 5, 1, 0)


In [10]:
# Tatil tarihlerini tanımlama

holiday_types = {"Yılbaşı"  : ["2016-01-01", "2017-01-01", "2018-01-01", "2019-01-01", "2020-01-01", "2021-01-01", "2022-01-01"],
                "23 Nisan"  : ["2016-04-23", "2017-04-23", "2018-04-23", "2019-04-23", "2020-04-23", "2021-04-23", "2022-04-23"],
                "1 Mayıs"   : ["2016-05-01", "2017-05-01", "2018-05-01", "2019-05-01", "2020-05-01", "2021-05-01", "2022-05-01"],
                "19 Mayıs"  : ["2016-05-19", "2017-05-19", "2018-05-19", "2019-05-19", "2020-05-19", "2021-05-19", "2022-05-19"],
                "30 Ağustos": ["2016-08-30", "2017-08-30", "2018-08-30", "2019-08-30", "2020-08-30", "2021-08-30", "2022-08-30"],
                "29 Ekim"   : ["2016-10-29", "2017-10-29", "2018-10-30", "2019-10-30", "2020-10-30", "2021-10-30", "2022-10-30"],
                
                 "Ramazan"  : [("2016-07-05", "2016-07-06", "2016-07-07"),
                               ("2017-06-25", "2017-06-26", "2017-06-27"),
                               ("2018-06-14", "2018-06-15", "2018-06-16"),
                               ("2019-06-04", "2019-06-05", "2019-06-16"),
                               ("2020-06-24", "2020-06-25", "2020-06-26"),
                               ("2021-05-13", "2021-05-14", "2021-05-15"),
                               ("2022-05-02", "2022-05-03", "2022-05-04")],
                 
                 "Kurban"   : [("2016-09-12", "2016-09-13", "2016-09-14", "2016-09-15"),
                               ("2017-09-01", "2017-09-02", "2017-09-03", "2017-09-04"),
                               ("2018-08-21", "2018-08-22", "2018-08-23", "2018-08-24"),
                               ("2019-08-11", "2019-08-12", "2019-08-13", "2019-08-14"),
                               ("2020-07-31", "2020-08-01", "2020-08-02", "2020-09-03"),
                               ("2021-07-20", "2021-07-21", "2021-07-22", "2021-07-23"),
                               ("2022-07-09", "2022-07-10", "2022-07-11", "2022-07-12")]}

In [11]:
# Veri kümesindeki tatilleri işaretleme

df["HolidayType"] = "weekday"

for holiday, dates in holiday_types.items():
    for date_range in dates:
        if isinstance(date_range, tuple):
            for date in date_range:
                df.loc[df["CheckInDate"] == date, "HolidayType"] = holiday
        else:
            df.loc[df["CheckInDate"]== date_range, "HolidayType"] = holiday

**Adım 2:** Satış ile Check-in tarihleri arasındaki farkı ele alarak "CustomerType" isimli bir değişken oluşturunuz. "SaleCheckInDayDiff" değişkeni, müşterinin Check-in tarihinden ne kadar önce satın alınımını tamamladığını gösterir. Aralıkları aşağıdaki gibi oluşturabilirsiniz:

Örneğin, **'0_7', 7_30', '30-90', '90_max'** gibi aralıkları kullanabilirsiniz. Bu aralıklar için **"Last Minuters", "Potential Planners", "Planners", "Early Bookers"** gibi isimleri kullanabilirsiniz.


In [12]:
# Müşteri türü için aralıkları tanımlama

bins = [-1, 7, 30, 90, df["SaleCheckInDayDiff"].max()]

# Müşteri türü için etiketleri tanımlama

labels = ["Son Dakika Rezervasyoncuları", "Potansiyel Planlayıcılar", "Planlayıcılar", "Erken Rezervasyon Yapanlar"]

# Rezervasyon günlerine göre müşterileri kategorize edelim

df["CustomerType"] = pd.cut(df["SaleCheckInDayDiff"], bins = bins, labels = labels)

In [13]:
df.head()

,SaleId,SaleDate,CheckInDate,Price,ConceptName,SaleCityName,CInDay,SaleCheckInDayDiff,Seasons,CheckInYear,CheckInMonth,CheckInDay,CheckInWeekday,IsWeekend,HolidayType,CustomerType
0,415122,2022-12-03,2022-12-03,79.30,Herşey Dahil,Antalya,Saturday,0,Low,2022,12,3,5,1,weekday,Son Dakika Rezervasyoncuları
1,415103,2022-12-03,2022-12-03,45.97,Yarım Pansiyon,Antalya,Saturday,0,Low,2022,12,3,5,1,weekday,Son Dakika Rezervasyoncuları
2,404034,2022-09-12,2022-09-13,77.84,Herşey Dahil,Antalya,Tuesday,1,High,2022,9,13,1,0,weekday,Son Dakika Rezervasyoncuları
3,415094,2022-12-03,2022-12-10,222.71,Yarım Pansiyon,İzmir,Saturday,7,Low,2022,12,10,5,1,weekday,Son Dakika Rezervasyoncuları
4,414951,2022-12-01,2022-12-03,140.48,Yarım Pansiyon,İzmir,Saturday,2,Low,2022,12,3,5,1,weekday,Son Dakika Rezervasyoncuları


In [14]:
df.shape

(59151, 16)

**Adım 3:** Kategorik değişkenler olan **'ConceptName', 'SaleCityName', 'CInDay', 'Seasons', 'HolidayType'** ve **CustomerType'** için one-hot encoding işlemi uygulayınız. 

In [15]:
df = pd.get_dummies(df, columns = ['ConceptName', 'SaleCityName', 'CInDay', 'Seasons', 'HolidayType', 'CustomerType'])

#### Görev 3: PCA ile Boyut Azaltma

**Adım 1:** Hedef değişken olan **'Price'** ile bağımsız değişkenleri ayırın.

In [16]:
# Özellikleri ve kategorik değişkenleri hazırlama

X = df.drop(["Price", "SaleId", "SaleDate", "CheckInDate"], axis = 1)

y = df["Price"]

**Adım 2:** Boyut azaltma işlemi için **'sklearn.decomposition'** modülündeki PCA fonksiyonunu kullanın ve Principal Component sayısını **5** olarak belirleyiniz.

**Adım 3:** Boyutu azaltılmış veriyi DataFrame veri tipine dönüştürünüz.

In [17]:
# PCA model oluşturma

pca = PCA(n_components = 5)

# Modeli fit etme

X_new = pca.fit_transform(X)

# Düşük boyutlu temsil edilen veriyi DataFrame'e dönüştürme

X_dimensional_reduced = pd.DataFrame(X_new)

#### Görev 4: Lineer Regresyon Modeli Kurma

**Adım 1:** Veriyi **'train_test_split'** fonksiyonunu kullanarak train ve test setlerine ayırınız. Test veri setinin boyutunu %20 olarak belirleyiniz.

**Adım 2:** **'sklearn.linear_model'** modülündeki 'LinearRegression' fonksiyonunu kullanarak modeli kurunuz ve eğitim seti için modele uydurunuz.

**Adım 3:** Test setini kullanarak tahminler yapın ve modelin başarısını **MSE, MAE, R2** gibi metriklerle değerlendiriniz.

In [18]:
# Veriyi eğitim ve test veri kümelerine ayırma

X_train, X_test, y_train, y_test = train_test_split(X_dimensional_reduced, y, test_size = 0.2, random_state = 42)

# Modeli başlatma ve eğitme

model = LinearRegression()
model.fit(X_train, y_train)

# Tahminler Yapma

y_pred = model.predict(X_test)

**Adım 4:** Modelin katsayılarını ekrana yazdırın.

In [19]:
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
print("R^2 Score:", r2_score(y_test, y_pred))

Mean Squared Error: 2686.7106775945367
Mean Absolute Error: 26.57458189373316
R^2 Score: 0.013357567952517302


**Adım 5:** Derslerde öğrendiğimiz **"en_kucuk_kareler_yontemi"** adlı fonksiyonu kullanarak modelin katsayılarını yeniden hesaplayın ve **LinearRegression** hazır fonksiyonu ile elde ettiğiniz katsayıları karşılaştırın. 